In [1]:
from bert.extract_feature import BertVector
bv = BertVector()

INFO:tensorflow:Using config: {'_model_dir': '../tmp', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
  allow_growth: true
}
graph_options {
  optimizer_options {
    global_jit_level: ON_1
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x12fa91c40>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Could not find trained model in model_dir: ../tmp, running initialization to predict.


In [2]:
import pickle
import pandas as pd

data = pd.read_pickle("./data0224.pkl")
print("Shape of data", data.shape) 
print("Number of rows : ", data.shape[0]) 
print("Number of columns : ", data.shape[1]) 

Shape of data (99, 7)
Number of rows :  99
Number of columns :  7


In [3]:
data.head()

,claim,source,category,times,text,label,Unnamed: 6
0,教你如何用肉眼就能看出這隻猪有没有餵瘦肉精！帮你到市場買猪肉時，你先看他的猪皮與肉中間這一塊...,cofacts,生活,NaN,教你如何用肉眼就能看出這隻猪有没有餵瘦肉精！\n帮你到市場買猪肉時，你先看他的猪皮與肉中間這...,1,NaN
1,指揮中心提醒，民眾出入醫院請佩戴口罩,中央衛生署,生活,1090127.0,中央流行疫情指揮中心表示，為防範中國大陸新型冠狀病毒肺炎疫情，提醒民眾及陪病家屬...,0,NaN
2,戒菸益健康、助防疫、愛地球！,中央衛生署,生活,1090421.0,4月22日是世界地球日50周年，今年可以思考用另一種方式愛護地球－戒菸！眾所皆知，戒菸有益健...,0,NaN
3,國民健康署有設置青少年網站性福ｅ學園網站,中央衛生署,生活,1090824.0,七夕佳節即將來臨，青少年相約出遊若缺少自我保護概念，可能陷於危險之中而不自知國民健康署提醒青...,0,NaN
4,健保卡科技防疫再進化，實名制口罩自動販賣系統在台北市試辦上線,中央衛生署,生活,1090411.0,口罩實名制販售通路再進化！衛生福利部、中央健保署與台北市政府攜手合作防疫，自4月12日起於台...,0,NaN


In [4]:
target_feature = 'claim'
data_set = data.filter([target_feature, 'label'])

In [5]:
print(data_set.head(10))

INFO:tensorflow:Done calling model_fn.
                                               claim  label
0  教你如何用肉眼就能看出這隻猪有没有餵瘦肉精！帮你到市場買猪肉時，你先看他的猪皮與肉中間這一塊...      1
1                                 指揮中心提醒，民眾出入醫院請佩戴口罩      0
2                                     戒菸益健康、助防疫、愛地球！      0
3                               國民健康署有設置青少年網站性福ｅ學園網站      0
4                     健保卡科技防疫再進化，實名制口罩自動販賣系統在台北市試辦上線      0
5                                降低風險！指揮中心公布社交距離注意事項      0
6                               防疫補償金可在2年內申請可多利用網路申辦      0
7                          防詐騙！指揮中心從未發最終通知，接獲不明郵件勿開啟      0
8                             科技防疫，再推疫止神通、雙向簡訊追蹤隔離檢疫      0
9                           享受生活促進經濟，業者、民眾齊響應防疫新生活運動      0
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [6]:
data_set_filtered = data_set[data_set[target_feature].notnull()]

In [7]:
import re
import demoji
demoji.download_codes()
for idx in range(0, data_set_filtered.shape[0]):
    data_set_filtered[target_feature].iloc[idx] = demoji.replace(data_set_filtered[target_feature].iloc[idx],"") # remove emojis
    data_set_filtered[target_feature].iloc[idx] = re.sub(r'http\S+', 'd', data_set_filtered[target_feature].iloc[idx]) # remove https link
    data_set_filtered[target_feature].iloc[idx] = re.sub(r'line:\S+', '', data_set_filtered[target_feature].iloc[idx]) # remove line link
    data_set_filtered[target_feature].iloc[idx] = re.sub(r"[\n\t\s]*", "", data_set_filtered[target_feature].iloc[idx]) # remove redundent space

... OK (Got response in 0.75 seconds)
Writing emoji data to /Users/Paul/.demoji/codes.json ...
... OK


In [8]:
data_set_filtered.insert(1, target_feature + "_vector", 0) 
data_set_filtered[target_feature + "_vector"] = data_set_filtered[target_feature + "_vector"].astype(object)
print(data_set_filtered.isnull().any())
print(data_set_filtered.shape)

claim           False
claim_vector    False
label           False
dtype: bool
(99, 3)


In [9]:
data_set_filtered_ws = data_set_filtered.copy()

In [10]:
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER
ws = WS("./tagger_data")
pos = POS("./tagger_data")
ner = NER("./tagger_data")

In [11]:
stop_words = open('./stop_words.txt', 'r')
lines = stop_words.readlines()
stop_words_dict = {}
for line in lines:
    line = line.replace('\n','')
    stop_words_dict[line] = 1
dictionary = construct_dictionary(stop_words_dict)

In [12]:
claim_ws = []
for claim in data_set_filtered_ws[target_feature]:
    sentence_list = [claim]
    word_sentence_list = ws(
        sentence_list,
        # sentence_segmentation = True, # To consider delimiters
        # segment_delimiter_set = {",", "。", ":", "?", "!", ";", "，"}, # This is the defualt set of delimiters
        recommend_dictionary = dictionary, # words in this dictionary are encouraged
        # coerce_dictionary = dictionary2, # words in this dictionary are forced
    )
    claim_ws.append(word_sentence_list[0])
    print(word_sentence_list[0])
data_set_filtered_ws[target_feature] = claim_ws

['教', '你', '如何', '用', '肉眼', '就', '能', '看出', '這', '隻', '猪', '有', '没', '有', '餵', '瘦肉精', '！', '帮', '你', '到', '市場', '買', '猪肉', '時', '，', '你', '先', '看', '他', '的', '猪皮', '與', '肉', '中間', '這', '一', '塊', '肥肉', '有', '幾', '公分', '！', '如果', '说', '中間', '這', '塊', '肥肉', '越', '少', '越', '薄', '，', '那', '就', '表示', '這', '隻', '豬', '有', '餵', '過', '瘦肉精', '正常', '的話', '猪皮', '跟', '瘦肉', '之間', '隔着', '這', '層', '肥肉', '的', '距離', '，', '約', '超過', '1', '公分', '之', '1', '公分', '半', '，', '有的', '甚至', '约两', '公分', '所謂', '瘦肉精', '就是', '要', '讓', '猪隻减', '少體内', '的', '肥肉', '，', '當', '猪', '的', '肥肉', '最', '明顯', '的', '就', '顯示', '在', '猪皮', '與', '身体', '瘦肉', '的', '中間', '這', '一', '層', '，', '這', '種', '判断', '準確率', '可', '達到', '90%', '!']
['指揮中心', '提醒', '，', '民眾', '出入', '醫院', '請', '佩戴', '口罩']
['戒菸', '益', '健康', '、', '助', '防疫', '、', '愛', '地球', '！']
['國民', '健康署', '有', '設置', '青少年', '網站性', '福', 'ｅ', '學園', '網站']
['健保卡', '科技', '防疫', '再', '進化', '，', '實名制', '口罩', '自動', '販賣', '系統', '在', '台北市', '試辦', '上線']
['降低', '風險', '！', '指揮中心', '公布', '社交距離', '注意', '事

In [13]:
max_len = 100
row_to_drop = []
for idx in range(0, data_set_filtered_ws.shape[0]):
    if(len(data_set_filtered_ws[target_feature].iloc[idx]) > max_len):
        row_to_drop.append(idx)
        print(idx)
data_set_filtered_ws = data_set_filtered_ws.drop(row_to_drop)
data_set_filtered = data_set_filtered.drop(row_to_drop)

0


In [14]:
# vectering sentence
for idx in range(0, data_set_filtered.shape[0]):
    sys.stdout.write('\r'+ "Implementing BERT vectoring {}% \n".format(round(100 * idx/data_set_filtered.shape[0], 2)))
    vec = bv.encode([data_set_filtered[target_feature].iloc[idx]])
    data_set_filtered[target_feature + "_vector"].iloc[idx] = vec

Implementing BERT vectoring 0.0% 
Implementing BERT vectoring 1.02% 
Implementing BERT vectoring 2.04% 
Implementing BERT vectoring 3.06% 
Implementing BERT vectoring 4.08% 
Implementing BERT vectoring 5.1% 
Implementing BERT vectoring 6.12% 
Implementing BERT vectoring 7.14% 
Implementing BERT vectoring 8.16% 
Implementing BERT vectoring 9.18% 
Implementing BERT vectoring 10.2% 
Implementing BERT vectoring 11.22% 
Implementing BERT vectoring 12.24% 
Implementing BERT vectoring 13.27% 
Implementing BERT vectoring 14.29% 
Implementing BERT vectoring 15.31% 
Implementing BERT vectoring 16.33% 
Implementing BERT vectoring 17.35% 
Implementing BERT vectoring 18.37% 
Implementing BERT vectoring 19.39% 
Implementing BERT vectoring 20.41% 
Implementing BERT vectoring 21.43% 
Implementing BERT vectoring 22.45% 
Implementing BERT vectoring 23.47% 
Implementing BERT vectoring 24.49% 
Implementing BERT vectoring 25.51% 
Implementing BERT vectoring 26.53% 
Implementing BERT vectoring 27.55% 
Imple

In [15]:
# vectering word by word
import sys

for idx in range(0, data_set_filtered_ws.shape[0]):
    sys.stdout.write('\r'+ "Implementing BERT vectoring {}% \n".format(round(100 * idx/data_set_filtered_ws.shape[0], 2)))
    title_list = []
    for word in data_set_filtered_ws[target_feature].iloc[idx]:
            vec = bv.encode([word])
            title_list.append(vec)
            data_set_filtered_ws[target_feature + "_vector"].iloc[idx] = title_list

Implementing BERT vectoring 0.0% 
Implementing BERT vectoring 1.02% 
Implementing BERT vectoring 2.04% 
Implementing BERT vectoring 3.06% 
Implementing BERT vectoring 4.08% 
Implementing BERT vectoring 5.1% 
Implementing BERT vectoring 6.12% 
Implementing BERT vectoring 7.14% 
Implementing BERT vectoring 8.16% 
Implementing BERT vectoring 9.18% 
Implementing BERT vectoring 10.2% 
Implementing BERT vectoring 11.22% 
Implementing BERT vectoring 12.24% 
Implementing BERT vectoring 13.27% 
Implementing BERT vectoring 14.29% 
Implementing BERT vectoring 15.31% 
Implementing BERT vectoring 16.33% 
Implementing BERT vectoring 17.35% 
Implementing BERT vectoring 18.37% 
Implementing BERT vectoring 19.39% 
Implementing BERT vectoring 20.41% 
Implementing BERT vectoring 21.43% 
Implementing BERT vectoring 22.45% 
Implementing BERT vectoring 23.47% 
Implementing BERT vectoring 24.49% 
Implementing BERT vectoring 25.51% 
Implementing BERT vectoring 26.53% 
Implementing BERT vectoring 27.55% 
Imple

In [16]:
print(data_set_filtered_ws.shape)
data_set_filtered_ws.to_pickle('data_title_ws_0224.pkl')

(98, 3)


In [17]:
print(data_set_filtered.shape)
data_set_filtered.to_pickle('data_title_sentence_0224.pkl')

(98, 3)


In [18]:
del data_set_filtered
del data_set_filtered_ws

In [19]:
!rm -r ./bert/tmp/